In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [ ]:
import os
import os.path as osp
import torch
import numpy as np
from tqdm import tqdm as tqdm
from matplotlib.pyplot import figure
from sklearn.decomposition import PCA
from wpca import WPCA, EMPCA
import pandas as pd
from scipy.optimize import curve_fit
import plotly.express as px
import math

from scipy.stats import norm
import matplotlib.mlab as mlab
import scipy.stats as scs
import matplotlib.pyplot as plt


import matplotlib.pyplot as plt

import mplhep as hep
import glob

In [ ]:
# Function to convert the number of the layer L to the z coordinate
convLtoZ = {1: 322.10272, 2: 323.0473, 3: 325.07275, 4: 326.01727, 5: 328.0428, 6: 328.98727, 7: 331.01276, 8: 331.9572, 9: 333.9828, 10: 334.92725, 
            11: 336.95273, 12: 337.89728, 13: 339.9228, 14: 340.86725, 15: 342.89273, 16: 343.8373, 17: 345.86276, 18: 346.80716, 19: 348.83282, 20: 349.77734, 
            21: 351.80276, 22: 352.74725, 23: 354.7728, 24: 355.71725, 25: 357.74274, 26: 358.6873, 27: 360.7128, 28: 361.65726, 29: 367.69904, 30: 373.149, 
            31: 378.59906, 32: 384.04898, 33: 389.499, 34: 394.94897, 35: 400.39902, 36: 405.84903, 37: 411.27457, 38: 416.7256, 39: 422.17126, 40: 427.62305, 
            41: 436.16428, 42: 444.7138, 43: 453.25305, 44: 461.8166, 45: 470.36862, 46: 478.9221, 47: 487.46515, 48: 496.02094, 49: 504.57083, 50: 513.1154}

def conversionLtoZ(layer):
    return convLtoZ[int(layer)]

# Load data

In [ ]:
raw_dir='/grid_mnt/data__data.polcms/cms/tarabini/GENPHOTESTPU2_noSmearing/step3/step4/'
fnamelist = [filepath for filepath in glob.glob(raw_dir+'data_eWeighted_*.pt')]
data_list_pho = []
for i in tqdm(fnamelist):
    idx = torch.load(i)
    data_list_pho.append(idx)

# Event display

In [ ]:
evt = 0

plt.style.use(hep.style.CMS)

#Prepare array
X = data_list_pho[evt].x
X = X.numpy()
X = np.concatenate((X,np.array([conversionLtoZ(i) for i in X[:,2:3]])[:, np.newaxis]), axis=1)
X[:,[2,4]] = X[:,[4,2]]
X[:,[3,4]] = X[:,[4,3]] #After this last swap the structure of X is the following [x,y,z,L,energy]
cleanpcaarr = data_list_pho[evt].ctrk

#Set centre of the PCA axis
maxx = data_list_pho[evt].centre[0]
maxy = data_list_pho[evt].centre[1]
maxl = data_list_pho[evt].centre[2]

#Computation of the gun particle direction
theta = 2*math.atan(math.exp(-data_list_pho[evt].geta))
phi = data_list_pho[evt].gphi
x = math.sin(theta)*math.cos(phi)
y = math.sin(theta)*math.sin(phi)
z = math.cos(theta)

fig, (ax0,ax1,ax2,ax3,ax4) = plt.subplots(1, 5, figsize=(42,17))

#ax0
ax0.scatter(X[:,0],X[:,3],s=X[:,4]*10,c='r')
ax0.set_xlabel('x', fontsize = 25, fontweight = 'bold', loc='center')
ax0.set_ylabel('Layer number', fontsize = 25, fontweight = 'bold', loc='center')
ax0.scatter(cleanpcaarr[:,0],cleanpcaarr[:,2],s=cleanpcaarr[:,3]*10,facecolors='deepskyblue', linewidths=2.5, color='b')
ax0.scatter(maxx, maxl, s=140,  marker='.', color ='black')
ax0.tick_params(axis='both', which='major', labelsize=15)
ax0.tick_params(axis='both', which='minor', labelsize=15)

#ax1
ax1.scatter(X[:,1],X[:,3],s=X[:,4]*10,c='r')
ax1.set_xlabel('y', fontsize = 25, fontweight = 'bold', loc='center')
ax1.set_ylabel('Layer number', fontsize = 25, fontweight = 'bold', loc='center')
ax1.scatter(maxy, maxl, s=140,  marker='.', color ='black')
ax1.scatter(cleanpcaarr[:,1],cleanpcaarr[:,2],s=cleanpcaarr[:,3]*10,facecolors='deepskyblue', linewidths=2.5, color='b')
ax1.tick_params(axis='both', which='major', labelsize=15)
ax1.tick_params(axis='both', which='minor', labelsize=15)


#ax2
ax2.scatter(X[:,0],X[:,1],s=X[:,4]*10,c='r')
ax2.set_xlabel('x', fontsize = 25, fontweight = 'bold', loc='center')
ax2.set_ylabel('y', fontsize = 25, fontweight = 'bold', loc='center')
ax2.scatter(maxx, maxy, s=140,  marker='.', color ='black')
ax2.set_ylim(ax2.get_ylim())
ax2.set_xlim(ax2.get_xlim())
ax2.arrow(*[maxx,maxy], data_list_pho[evt].axis[0] * 800, data_list_pho[evt].axis[1] * 800, color='tab:orange') 
ax2.arrow(*[maxx,maxy], data_list_pho[evt].axis[0] * -800, data_list_pho[evt].axis[1] * -800, color='tab:orange') 
ax2.arrow(0, 0, x * 800, y * 800, color='black')
ax2.scatter(cleanpcaarr[:,0],cleanpcaarr[:,1],s=cleanpcaarr[:,3]*10,facecolors='deepskyblue', linewidths=2.5, color='b')
ax2.tick_params(axis='both', which='major', labelsize=15)
ax2.tick_params(axis='both', which='minor', labelsize=15)

#ax3
ax3.scatter(X[:,0],X[:,2],s=X[:,4]*10,c='r')
ax3.set_xlabel('x', fontsize = 25, fontweight = 'bold', loc='center')
ax3.set_ylabel('z', fontsize = 25, fontweight = 'bold', loc='center')
ax3.scatter(maxx, conversionLtoZ(maxl), s=140,  marker='.', color ='black')
ax3.set_ylim(ax3.get_ylim())
ax3.set_xlim(ax3.get_xlim())
ax3.arrow(*[maxx,conversionLtoZ(maxl)], data_list_pho[evt].axis[0] * 800, data_list_pho[evt].axis[2] * 800, color='tab:orange') 
ax3.arrow(*[maxx,conversionLtoZ(maxl)], data_list_pho[evt].axis[0] * -800, data_list_pho[evt].axis[2] * -800, color='tab:orange') 
ax3.arrow(0, 0, x * 400, z * 400, color='black')
ax3.scatter(cleanpcaarr[:,0],cleanpcaarr[:,4],s=cleanpcaarr[:,3]*10,facecolors='deepskyblue', linewidths=2.5, color='b')
ax3.tick_params(axis='both', which='major', labelsize=15)
ax3.tick_params(axis='both', which='minor', labelsize=15)

#ax4
ax4.scatter(X[:,1],X[:,2],s=X[:,4]*10,c='r')
ax4.set_xlabel('y', fontsize = 25, fontweight = 'bold', loc='center')
ax4.set_ylabel('z', fontsize = 25, fontweight = 'bold', loc='center')
ax4.scatter(maxy, conversionLtoZ(maxl), s=140,  marker='.', color ='black')
ax4.set_ylim(ax4.get_ylim())
ax4.set_xlim(ax4.get_xlim())
ax4.arrow(*[maxy,conversionLtoZ(maxl)], data_list_pho[evt].axis[1] * 800, data_list_pho[evt].axis[2] * 800, color='tab:orange') 
ax4.arrow(*[maxy,conversionLtoZ(maxl)], data_list_pho[evt].axis[1] * -800, data_list_pho[evt].axis[2] * -800, color='tab:orange') 
ax4.arrow(0, 0, y * 400, z * 400, color='black')
ax4.scatter(cleanpcaarr[:,1],cleanpcaarr[:,4],s=cleanpcaarr[:,3]*10,facecolors='deepskyblue', linewidths=2.5, color='b')
ax4.tick_params(axis='both', which='major', labelsize=15)
ax4.tick_params(axis='both', which='minor', labelsize=15)


# Dataframes

In [ ]:
def computeEta(axis):
    return -math.log(math.tan(math.acos(axis[2])/2))

### ---- After cleaning

In [ ]:
datafr_after = {'true_en': [float(data_list_pho[i].en[0]) for i in range(len(data_list_pho))],
                'true_eta': [float(data_list_pho[i].geta[0]) for i in range(len(data_list_pho))], 
                'eta_PCA': [computeEta(data_list_pho[i].axis) for i in range(len(data_list_pho))],
                'cleaned_en': [np.sum(np.array(data_list_pho[i].ctrk[:,3])) for i in range(len(data_list_pho))],
                'reco_en': [np.sum(np.array(data_list_pho[i].x[:,3])) for i in range(len(data_list_pho))]}
datafr_after = pd.DataFrame(data=datafr_after)
datafr_after['true_pt'] = [en/math.cosh(eta) for en,eta in zip(datafr_after['true_en'], datafr_after['true_eta'])]

### ---- Before cleaning

In [ ]:
# Before cleaning
datafr_before = pd.DataFrame(columns=['eta_PCA'])

for evt in tqdm(range(len(data_list_pho))):
    X = data_list_pho[evt].x
    X = X.numpy()
    X = np.concatenate((X,np.array([conversionLtoZ(i) for i in X[:,2:3]])[:, np.newaxis]), axis=1)
    X[:,[2,4]] = X[:,[4,2]]
    X[:,[3,4]] = X[:,[4,3]] #After this last swap the structure of X is the following [x,y,z,L,energy]
    X = np.concatenate((X,X[:,4][:, np.newaxis]), axis=1)
    X = np.concatenate((X,X[:,4][:, np.newaxis]), axis=1)
    
    pca = WPCA(n_components=3)
    pca.fit(X[:,:3], weights = X[:,4:])
    
    eta_PCA = -math.log(math.tan(math.acos(*pca.components_[0,[2]])/2))
    
    datafr_before = datafr_before.append({'true_en':float(data_list_pho[evt].en[0]),
                                          'true_eta':float(data_list_pho[evt].geta[0]),
                                          'eta_PCA':eta_PCA}, ignore_index=True)        

datafr_before['true_pt'] = [en/math.cosh(eta) for en,eta in zip(datafr_before['true_en'], datafr_before['true_eta'])]

# DeltaEta

In [ ]:
fig = plt.figure(figsize=(12,7), dpi=80)
plt.style.use(hep.style.CMS)
hep.cms.label(llabel='Phase-II Simulation Preliminary',rlabel='')

pt_cut=[20,60]
eta_cut=[2,2.5]

sel = (datafr_after.true_pt>=pt_cut[0]) & (datafr_after.true_pt<=pt_cut[1]) & (datafr_after.true_eta>=eta_cut[0]) & (datafr_after.true_eta<=eta_cut[1])
etaDiff_after = np.array(abs(datafr_after[sel]['true_eta'] - datafr_after[sel]['eta_PCA']))
h_after, bins_after = np.histogram(etaDiff_after, range = [0,0.25], bins=55, density=True)
hep.histplot(h_after,bins_after,label = 'After cleaning', histtype = 'step', color = 'tab:red', linewidth=2.5)

sel = (datafr_before.true_pt>pt_cut[0]) & (datafr_before.true_pt<pt_cut[1]) & (datafr_before.true_eta>eta_cut[0]) & (datafr_before.true_eta<eta_cut[1])
etaDiff_before = np.array(abs(datafr_before[sel]['true_eta'] - datafr_before[sel]['eta_PCA']))
h_before, bins_before = np.histogram(etaDiff_before, range = [0,0.25], bins=55, density=True)
hep.histplot(h_before,bins_before, label = 'Before cleaning', histtype = 'step', color = 'black', linewidth=2.5)

plt.ylabel('a.u.', fontsize = 25, loc='center')
plt.legend(title=r'$\bf{PU = 200}$')._legend_box.align = "left"
plt.text(0.147,20,s=r'$\bf{'+str(eta_cut[0])+'<|\eta_{GEN}|<'+str(eta_cut[1])+'}$', size='small')
plt.text(0.147,17,s=r'$\bf{20<pT_{GEN}< 60}$ $\bf{[GeV]}$', size='small')
plt.xlim([0,0.22])
plt.xlabel('$|\eta^{dir}_{PCA} - \eta^{dir}_{GEN}|$', fontsize = 25, loc='center')
plt.savefig('eta_diff.pdf', dpi=400, bbox_inches='tight')
plt.show()


fig = plt.figure(figsize=(13,8), dpi=80)
plt.style.use(hep.style.CMS)
hep.cms.label(llabel='Phase-II Simulation Preliminary',rlabel='')

sel = (datafr_after.true_pt>pt_cut[0]) & (datafr_after.true_pt<pt_cut[1]) & (datafr_after.true_eta>eta_cut[0]) & (datafr_after.true_eta<eta_cut[1])
etaDiff_after = np.array(abs(datafr_after[sel]['true_eta'] - datafr_after[sel]['eta_PCA']))
h_after, bins_after = np.histogram(etaDiff_after, range = [0,0.8], bins=20, density =True)
hep.histplot(h_after,bins_after,label = 'After cleaning', histtype = 'step', color = 'tab:red', linewidth=2.5)

sel = (datafr_before.true_pt>pt_cut[0]) & (datafr_before.true_pt<pt_cut[1]) & (datafr_before.true_eta>eta_cut[0]) & (datafr_before.true_eta<eta_cut[1])
etaDiff_before = np.array(abs(datafr_before[sel]['true_eta'] - datafr_before[sel]['eta_PCA']))
h_before, bins_before = np.histogram(etaDiff_before, range = [0,0.8], bins=20, density =True)
hep.histplot(h_before,bins_before, label = 'Before cleaning', histtype = 'step', color = 'black', linewidth=2.5)

plt.ylabel('a.u.', fontsize = 25, loc='center')
plt.legend(title=r'$\bf{PU = 200}$')._legend_box.align = "left"
plt.yscale('log')
plt.xlim([0,0.8])
plt.xlabel('$|\eta^{dir}_{PCA} - \eta^{dir}_{GEN}|$', fontsize = 25, loc='center')
plt.savefig('eta_diff_log.pdf', dpi=400, bbox_inches='tight')
plt.show()


# Kinematics plots

In [ ]:
def plotEnergy(df, angles):
    figure(figsize=(12, 6), dpi=100)
    hh = plt.hist2d(df['true_energy'], df['angle'], range=[[10,1000], angles], bins=40, cmap=plt.cm.Reds)
    plt.colorbar(hh[3])
    plt.xlabel('True energy (GeV)')
    plt.ylabel('Angle between axes (°)')
    plt.show()
    
def plotDeltaPhi(df, angles):
    figure(figsize=(12, 6), dpi=100)
    hh = plt.hist2d(df['true_energy'], df['true_phi'] - df['phi_PCA'], range=[[10,1000], angles], bins=40, cmap=plt.cm.Reds)
    plt.axhline(y=0, c='black')
    plt.colorbar(hh[3])
    plt.xlabel('True energy (GeV)')
    plt.ylabel('PhiGun - PhiPCA (°)')
    plt.show()
    
def plotDeltaEta(df, angles):
    figure(figsize=(12, 6), dpi=100)
    plt.style.use(hep.style.CMS)
    hep.cms.label(rlabel = '')
    hh = plt.hist2d(df['true_en'], df['true_eta'] - df['eta_PCA'], range=[[10,1000], angles], bins=50, cmap=plt.cm.Reds)
    plt.colorbar(hh[3])
    plt.axhline(y=0, c='black')
    plt.xlabel('True energy (GeV)')
    plt.ylabel('EtaGun - EtaPCA')
    plt.show()
    
def plotEta(df, angles):
    figure(figsize=(12, 6), dpi=100)
    hh = plt.hist2d(df['true_eta'], df['angle'], range=[[1.65,2.75], angles], bins=40, cmap=plt.cm.Reds)
    plt.colorbar(hh[3])
    plt.xlabel('True eta')
    plt.ylabel('Angle between axes (°)')
    plt.show()

In [ ]:
# plotEnergy(datafr_wt, [0,4])
# plotDeltaPhi(datafr_wt, [-0.1,0.1])
plotDeltaEta(datafr_after, [-0.15,0.15])

# Reconstructed vs Cleaned vs True energies

In [ ]:
def plotEnergies(df):
    trE = np.array(df['true_en'])
    clE = np.array(df['cleaned_en'])
    recE = np.array(df['reco_en'])

    CleanRec = (clE-recE)*100/recE
    RecTrue = (recE-trE)*100/trE
    CleanTrue = (clE-trE)*100/trE
    
    figure(figsize=(15, 10), dpi=80)
    plt.hist(CleanTrue, range=[-100,100],bins=100,label='(cleaned-true)*100/true',histtype='step',linewidth=1.5)
    plt.hist(RecTrue, range=[-100,100],bins=100,label='(reconstructed-true)*100/true',histtype='step',linewidth=1.5)
    plt.hist(CleanRec, range=[-100,100],bins=100,label='(cleaned-reconstructed)*100/reconstructed',histtype='step',linewidth=1.5)
    plt.title("200 PU")
    plt.legend()
    plt.yscale('log')
    plt.grid()
    plt.savefig('energies.pdf', dpi=400, bbox_inches='tight')
    plt.show()

In [ ]:
plotEnergies(datafr_after)

# Energies plots

In [ ]:
# Energy bins
bins =  np.linspace(0,200,11)
bins2 = np.linspace(250,500,6)
bins = np.append(bins,bins2)
bins = np.append(bins,[600,700,800,900,1000])
print(bins)

# Useful for plots
bins_mean = [(bins[i]+bins[i+1])/2 for i in range(len(bins)-1)]

In [ ]:
# Define bin boundaries with same numbers of events
def histedges_equalN(x, nbin):
    npt = len(x)
    return np.interp(np.linspace(0, npt, nbin + 1),
                     np.arange(npt),
                     np.sort(x))


n, bins_eta, patches = plt.hist(datafr_after['true_eta'], histedges_equalN(datafr_after['true_eta'], 8))
print(bins_eta)
print(n)

In [ ]:
def plotpredoverlay(axs,pred,true,e1,e2,text,index):
    
    fracarr = (pred - true)/true
    bin_heights, bin_borders, _ = axs.hist(fracarr,range=xlim[index], bins=60, label=text,alpha=0.5)

    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2

#     x_interval_for_fit = np.linspace(bin_borders[0], bin_borders[-1], 100)
#     #axs.plot(x_interval_for_fit, gaussian(x_interval_for_fit, *popt), label='fit'+text)
#     axs.legend()


    axs.set_xlabel('pred - true / true')
    axs.set_ylabel('counts')
    #axs.set_yscale('log')
    #plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
    
    #axs.set_title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f,\ E=$%i to %i' %(popt[0], popt[2],e1,e2))
    axs.set_title(r'E=%i GeV to %i GeV' %(e1,e2))
    axs.grid(True)
    axs.set_xlim(xlim[index])
#    plt.show()
#     return [popt[0], popt[2],perr[0], perr[2]]
    return 1

def plotdistoverlay(df,pred,pred2,true,text1,text2,eta_boundary=[]):
    

    fig, axs = plt.subplots(3,7, figsize=(74, 30), facecolor='w', edgecolor='k')
    axs = axs.ravel()
    
    if eta_boundary: 
        print(eta_boundary[0], eta_boundary[1])

    for i in tqdm(range (bins.size - 1)):
        sel = (df.true_en >= bins[i]) & (df.true_en < bins[i+1])
        if eta_boundary:
            sel &= (df.true_eta >= eta_boundary[0]) & (df.true_eta < eta_boundary[1])
        preda = df[pred][sel]
        pred2a = df[pred2][sel]
        truea = df[true][sel]
#         preda = pred[(true >bins[i]) & (true <bins[i+1]) ]
#         pred2a = pred2[(true >bins[i]) & (true <bins[i+1]) ]
#         truea = true[(true >bins[i]) & (true <bins[i+1]) ]
        vals = plotpredoverlay(axs[i],preda,truea,bins[i],bins[i+1],text1,i)
        vals2 = plotpredoverlay(axs[i],pred2a,truea,bins[i],bins[i+1],text2,i)
#     plt.savefig('energy_plot.pdf', dpi=400, bbox_inches='tight')
    plt.show()

In [ ]:
#Limit for x-axis
xlim=[[-0.8,0.8], #1
      [-0.8,0.8], #2
      [-0.3,0.5], #3
      [-0.2,0.2], #4
      [-0.25,0.5], #5
      [-0.25,0.5], #6
      [-0.25,0.3], #7
      [-0.25,0.3], #8
      [-0.25,0.3], #9
      [-0.25,0.3], #10
      [-0.25,0.25], #11
      [-0.25,0.25], #12
      [-0.25,0.25], #13
      [-0.25,0.25], #14
      [-0.25,0.25], #15
      [-0.25,0.25], #16
      [-0.25,0.25], #17
      [-0.25,0.25], #18
      [-0.25,0.25], #19
      [-0.25,0.25], #20
      [-0.25,0.25], #21
      [-0.25,0.25], #22
      [-0.25,0.25], #23
      [-0.25,0.25], #24
      [-0.25,0.25], #25
     ]


In [ ]:
# Plot for presentation by Shamik
fig = plt.figure(figsize=(10,8), dpi=80)
plt.style.use(hep.style.CMS)
# hep.cms.label(rlabel = '')

sel = (datafr_after.true_en >= 60) & (datafr_after.true_en < 80)
pred = datafr_after['reco_en'][sel]
pred2 = datafr_after['cleaned_en'][sel]
true = datafr_after['true_en'][sel]
    
corr = 1/0.9325
# corr=1

rangea = [0.8, 1.3]

fracarr = pred*corr/true
H = np.histogram(fracarr,range=rangea, bins=30, density=True)
hep.histplot(H[0],H[1], alpha=0.5, label='Before cleaning', histtype='fill')
# print(H[1])
# print(H[1][np.where(H[0]==np.max(H[0]))] + (H[1][3] - H[1][2])/2)
# plt.axvline(x=H[1][np.where(H[0]==np.max(H[0]))] + (H[1][3] - H[1][2])/2)
fracarr = pred2*corr/true
H = np.histogram(fracarr,range=rangea, bins=30, density=True)
hep.histplot(H[0],H[1],alpha=0.5, label='After cleaning', histtype='fill', color='tab:red')
hep.cms.label(llabel='Phase-II Simulation Preliminary',rlabel='')
plt.xlim(rangea)
plt.legend(title=r'$\bf{PU = 200}$')._legend_box.align = "left"


# plt.legend()

plt.ylabel('a.u.', loc='center')
plt.xlabel('$E_{trackster} / E_{GEN}$', loc='center')

    
# plt.savefig('energy_plot_forShamik.pdf', dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# plt.rcParams.update({'font.size': 22})
# plt.rc('xtick', labelsize=22)
# plt.rc('ytick', labelsize=22)
plotdistoverlay(datafr_after,'reco_en','cleaned_en','true_en','reco','cleaned')
# plt.rcParams.update(plt.rcParamsDefault)
# plt.rc('font', **font)

In [ ]:
bins_eta = [1.65498078,1.73172045,1.81604886,1.91224909,2.02308965,2.15197039,2.30686784,2.5010618,2.75467777]

bins =  np.linspace(0,200,11)
bins2 = np.linspace(250,500,6)
bins = np.append(bins,bins2)
bins = np.append(bins,[600,700,800,900,1000])
print(bins)
bins_mean = [(bins[i]+bins[i+1])/2 for i in range(len(bins)-1)]
print(bins_mean)

# plt.rcParams.update({'font.size': 22})
# plt.rc('xtick', labelsize=22)
# plt.rc('ytick', labelsize=22)
for i in range(len(bins_eta)-1):
    plotdistoverlay(datafr_after,'reco_en','cleaned_en','true_en','reco','cleaned', [bins_eta[i],bins_eta[i+1]])
# plt.rcParams.update(plt.rcParamsDefault)
# plt.rc('font', **font)

# Resolution

In [ ]:
from numpy import mean, sqrt, square

In [ ]:
# RMS
def enRes(df, _bins, _bins_mean):
    y_axis_cleaned = []
    y_axis_err_cleaned = []
    for index_b in range(len(bins)-1):
        cleanEnergy = df['cleaned_en'][(df.true_en > _bins[index_b]) & (df.true_en < _bins[index_b+1])]
        recoEnergy = df['reco_en'][(df.true_en > _bins[index_b]) & (df.true_en < _bins[index_b+1])]
#         ratio_true_clean = (cleanEnergy/recoEnergy)-1
        ratio_true_clean = (cleanEnergy-recoEnergy)/recoEnergy
#         plt.hist(ratio_true_clean, bins=40, range=[-0.2,0])
#         plt.axvline(x=np.mean(ratio_true_clean), c='black')
#         plt.show()
        y_axis_cleaned.append(np.mean(ratio_true_clean))
#         y_axis_err_cleaned.append(np.std(ratio_true_clean))
        y_axis_err_cleaned.append(sqrt(mean(square(ratio_true_clean))))

    fig = plt.figure(figsize=(10,7), dpi=85)
    plt.title('CMS', loc='left', family='sans-serif', weight='bold')
    plt.text(100,0.213,'Phase-II Simulation Preliminary', style='italic', family='sans-serif')
#     plt.errorbar(_bins_mean, y_axis_cleaned, yerr=y_axis_err_cleaned, color='tab:blue', capsize=3, fmt='o')
    plt.errorbar(_bins_mean, y_axis_cleaned, yerr=y_axis_err_cleaned, color='tab:blue', fmt='o', ecolor='lightgray', elinewidth=6, capsize=0)
    plt.ylabel('($E_{clean}/E_{trackster}) - 1$', loc='center')
    plt.xlabel('$E_{GEN}$ [GeV]', loc='center')
    plt.ylim([-0.4,0.2])
    plt.axhline(0,color='black',ls='--')
    plt.legend(title=r'$\bf{PU = 200}$')._legend_box.align = "left"
#     plt.savefig('energy_resolution_diff.pdf', dpi=400, bbox_inches='tight')
    plt.show()


In [ ]:
# Energy bins
bins =  np.linspace(0,200,11)
bins2 = np.linspace(250,500,6)
bins = np.append(bins,bins2)
bins = np.append(bins,[600,700,800,900,1000])
print(bins)

# Useful for plots
bins_mean = [(bins[i]+bins[i+1])/2 for i in range(len(bins)-1)]

enRes(datafr_after, bins, bins_mean)

# Resolution (in eta bins)

In [ ]:
# Define bin boundaries with same numbers of events
def histedges_equalN(x, nbin):
    npt = len(x)
    return np.interp(np.linspace(0, npt, nbin + 1),
                     np.arange(npt),
                     np.sort(x))

def enResEta(df):
#     plt.rcParams.update({'font.size': 25})
#     plt.rc('xtick', labelsize=25)
#     plt.rc('ytick', labelsize=25)

    fig, axs = plt.subplots(2,4, figsize=(75, 22), facecolor='w', edgecolor='k')
    fig_bis, axs_bis = plt.subplots(2,4, figsize=(75, 22), facecolor='w', edgecolor='k')
    axs = axs.ravel()
    axs_bis = axs_bis.ravel()
    
    bins_eta = histedges_equalN(df['true_eta'], 8)
    print(bins_eta)

    for index_eta in range(len(bins_eta)-1):
        y_axis = []
        y_axis_err = []
        y_axis_cleaned = []
        y_axis_err_cleaned = []
        for index_b in range(len(bins)-1):
            sel = (df.true_en >= bins[index_b]) & (df.true_en < bins[index_b+1]) & (df.true_eta >= bins_eta[index_eta]) & (df.true_eta < bins_eta[index_eta+1])
            ratio_true_reco = df['reco_en'][sel] / df['true_en'][sel]
            y_axis.append(np.mean(ratio_true_reco))
            y_axis_err.append(np.std(ratio_true_reco))
            ratio_true_clean = df['cleaned_en'][sel] / df['true_en'][sel]
            y_axis_cleaned.append(np.mean(ratio_true_clean))
            y_axis_err_cleaned.append(np.std(ratio_true_clean))


        axs[index_eta].set_title(str('%.2f'%bins_eta[index_eta])+' < |$\eta$| < '+str('%.2f'%bins_eta[index_eta+1]), fontsize=25)
        axs[index_eta].errorbar(bins_mean, y_axis, yerr=y_axis_err, capsize=3, label = '$E_{reco} =$ w/o cleaning')
        axs[index_eta].errorbar(bins_mean, y_axis_cleaned, yerr=y_axis_err_cleaned, capsize=3, label = '$E_{reco} =$ after cleaning')
        axs[index_eta].set_xlabel('$E_{true}$ [GeV]', fontsize=25)
        axs[index_eta].set_ylabel('$E_{reco}/E_{true}$', fontsize=25)
        axs[index_eta].legend()

        axs_bis[index_eta].set_title(str('%.2f'%bins_eta[index_eta])+' < |$\eta$| < '+str('%.2f'%bins_eta[index_eta+1]), fontsize=25)
        axs_bis[index_eta].errorbar(bins_mean, y_axis, label = '$E_{reco} =$ w/o cleaning')
        axs_bis[index_eta].errorbar(bins_mean, y_axis_cleaned, label = '$E_{reco} =$ after cleaning')
        axs_bis[index_eta].set_xlabel('$E_{true}$ [GeV]', fontsize=25)
        axs_bis[index_eta].set_ylabel('$E_{reco}/E_{true}$', fontsize=25)
        axs_bis[index_eta].legend()
    plt.show()
#     plt.rcParams.update(plt.rcParamsDefault)
#     plt.rc('font', **font)

In [ ]:
enResEta(datafr_after)

# Resolution (same plot)

In [ ]:
def enResEta_samePlot(df):
    figure(figsize=(10, 5), dpi=80)
    
    bins_eta = histedges_equalN(df['true_eta'], 4)
    print(bins_eta)

    for index_eta in range(len(bins_eta)-1):
        y_axis = []
        y_axis_err = []
        y_axis_cleaned = []
        y_axis_err_cleaned = []
        for index_b in range(len(bins)-1):
            sel = (df.true_en >= bins[index_b]) & (df.true_en < bins[index_b+1]) & (df.true_eta >= bins_eta[index_eta]) & (df.true_eta < bins_eta[index_eta+1])
            ratio_true_reco = df['reco_en'][sel] / df['true_en'][sel]
            y_axis.append(np.mean(ratio_true_reco))
            y_axis_err.append(np.std(ratio_true_reco))
            ratio_true_clean = df['cleaned_en'][sel] / df['true_en'][sel]
            y_axis_cleaned.append(np.mean(ratio_true_clean))
            y_axis_err_cleaned.append(np.std(ratio_true_clean))

        err = plt.errorbar(bins_mean, y_axis, linestyle='dashed')
        plt.errorbar(bins_mean, y_axis_cleaned, color = err[0].get_color(),label = str('%.2f'%bins_eta[index_eta])+' < |$\eta$| < '+str('%.2f'%bins_eta[index_eta+1]))
    plt.xlabel('$E_{true}$ [GeV]')
    plt.ylabel('$E_{reco}/E_{true}$')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left',borderaxespad=0.,ncol=1)
    plt.show()

In [ ]:
enResEta_samePlot(datafr_after)